<a href="https://colab.research.google.com/github/galexbh/categorical-data-transformations/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Targets
- According to the adidas dataset, for categorical columns, find -those columns that have more than one level, and that at least one of those levels represents more or equal to 98%. For the columns that meet that premise, they will be removed from the array of valid columns. 

- According to the adidas dataset, for the categorical columns, find those columns that have between 3-6 levels. The level with the lowest frequency will be replaced by the word Others, once you have that rule apply it to the original dataset, as it was done with the exercise done in class.

In [71]:
import pandas as pd
import numpy as np

In [72]:
df_adidas = pd.read_csv('/content/drive/MyDrive/UNAH/IA/homework/adidas.csv')

In [73]:
df_adidas.head(5)

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,source_website,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at
0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,adidas United States,https://www.adidas.com,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255
1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,adidas United States,https://www.adidas.com,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830
2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,adidas United States,https://www.adidas.com,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834
3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,adidas United States,https://www.adidas.com,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054
4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,adidas United States,https://www.adidas.com,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680


In [74]:
print('size: ', df_adidas.shape[0])
print('columns: ', df_adidas.columns)

size:  845
columns:  Index(['url', 'name', 'sku', 'selling_price', 'original_price', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language',
       'average_rating', 'reviews_count', 'crawled_at'],
      dtype='object')


In [75]:
cols = ['url', 'name', 'sku', 'currency',
       'availability', 'color', 'category', 'source', 'source_website',
       'breadcrumbs', 'description', 'brand', 'images', 'country', 'language']

In [76]:
df_copy_adidas = df_adidas.copy()

valid_columns = []
columns_many_levels = []
columns_one_level = []

size_df_adidas = df_adidas.shape[0]
threshold = .98

for c_ in cols:
    levels = df_copy_adidas.loc[ : , [c_]].drop_duplicates()
    uniquevalues = levels.shape[0]
    
    if uniquevalues == 1:
        columns_one_level.append(c_)
    elif (uniquevalues / size_df_adidas) > threshold:
        columns_many_levels.append(c_)
    else:
        valid_columns.append(c_)

In [77]:
print('valid columns: ', valid_columns)
print('columns many levels: ', columns_many_levels)

# They do not contribute anything because they have the same value
print('columns one level: ', columns_one_level)

valid columns:  ['name', 'availability', 'color', 'category', 'breadcrumbs', 'description']
columns many levels:  ['url', 'sku', 'images']
columns one level:  ['currency', 'source', 'source_website', 'brand', 'country', 'language']


In [78]:
columns_to_delete = []

for c_ in valid_columns:
    levels = df_copy_adidas.groupby( [c_], as_index = False )\
    .agg({'sku' : ['count']})
    levels.columns = levels.columns.droplevel(1)
    levels['condition'] = levels\
    .apply(lambda r: r['sku'] / size_df_adidas, axis=1)
    if(( levels['condition'].max()) >= threshold):
        columns_to_delete.append(c_) 

for c_ in columns_to_delete:
    if(c_ in (valid_columns)):
       valid_columns.remove(c_)
    

In [79]:
print('new valid colums: ', valid_columns)

new valid colums:  ['name', 'color', 'category', 'breadcrumbs', 'description']


In [87]:
lower_threshold = 3
upper_threshold = 6

for c_ in valid_columns:
    levels = df_copy_adidas.groupby([c_], as_index = False)\
    .agg({'sku' : ['count']})
    levels.columns = levels.columns.droplevel(1)

    if((uniquevalues <= lower_threshold) & (uniquevalues >= upper_threshold)):
        column_selected = levels.loc[: , [c_, 'sku']]

#print(column_selected)

column_selected.columns = ['category','frequency']

column_selected = column_selected.sort_values(['frequency'], ascending=False)
column_selected

,category,frequency
2,Shoes,Shoes
1,Clothing,Others
0,Accessories,Accessories


In [81]:
column_selected['level'] = 'Others'
value_min = column_selected['frequency'].min()

column_selected['validation'] = column_selected\
.apply(lambda r: r['level'] if r['frequency'] == value_min else r['category'], axis=1)

column_selected

,category,frequency,level,validation
2,Shoes,Shoes,Others,Shoes
0,Accessories,Others,Others,Accessories
1,Clothing,Clothing,Others,Others


In [82]:
column_selected = column_selected.loc[ : , ['category','validation']]

df_copy_adidas = df_copy_adidas.merge(column_selected, on='category', how='left')

df_copy_adidas.head(5)

,url,name,sku,selling_price,original_price,currency,availability,color,category,source,...,breadcrumbs,description,brand,images,country,language,average_rating,reviews_count,crawled_at,validation
0,https://www.adidas.com/us/beach-shorts/FJ5089....,Beach Shorts,FJ5089,40,NaN,USD,InStock,Black,Clothing,adidas United States,...,Women/Clothing,Splashing in the surf. Making memories with yo...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.5,35,2021-10-23 17:50:17.331255,Others
1,https://www.adidas.com/us/five-ten-kestrel-lac...,Five Ten Kestrel Lace Mountain Bike Shoes,BC0770,150,NaN,USD,InStock,Grey,Shoes,adidas United States,...,Women/Shoes,Lace up and get after it. The Five Ten Kestrel...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.8,4,2021-10-23 17:50:17.423830,Shoes
2,https://www.adidas.com/us/mexico-away-jersey/G...,Mexico Away Jersey,GC7946,70,NaN,USD,InStock,White,Clothing,adidas United States,...,Kids/Clothing,"Clean and crisp, this adidas Mexico Away Jerse...",adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.9,42,2021-10-23 17:50:17.530834,Others
3,https://www.adidas.com/us/five-ten-hiangle-pro...,Five Ten Hiangle Pro Competition Climbing Shoes,FV4744,160,NaN,USD,InStock,Black,Shoes,adidas United States,...,Five Ten/Shoes,The Hiangle Pro takes on the classic shape of ...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,3.7,7,2021-10-23 17:50:17.615054,Shoes
4,https://www.adidas.com/us/mesh-broken-stripe-p...,Mesh Broken-Stripe Polo Shirt,GM0239,65,NaN,USD,InStock,Blue,Clothing,adidas United States,...,Men/Clothing,Step up to the tee relaxed. This adidas golf p...,adidas,"https://assets.adidas.com/images/w_600,f_auto,...",USA,en,4.7,11,2021-10-23 17:50:17.702680,Others


In [83]:
print(df_copy_adidas.shape[0])

845
